# Import All Libaries

In [1]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
ensure_pyterrier_is_loaded()

import pyterrier as pt
from jnius import autoclass



# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


# Load the Dataset and the Index

In [2]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
dataset = 'ir-acl-anthology-20240504-training'
pt_dataset = pt.get_dataset(f'irds:ir-benchmarks/{dataset}')
#bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', dataset)

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

## Stopwords

In [3]:
def create_index(documents, stopwords):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stopwords=stopwords)
    index_ref = indexer.index(documents)
    return pt.IndexFactory.of(index_ref)

chatGPTStopwords =[
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 
    'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 
    'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 
    'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 
    'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 
    'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 
    'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 
    'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 
    'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 
    'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 
    'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 
    'will', 'just', 'don', 'should', 'now'
]
##Von Github https://github.com/igorbrigadir/stopwords/tree/master
##Natural Language Toolkit (NLTK)
ntlkStopwords = [
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", 
    "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", 
    "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", 
    "theirs", "themselves", "what", "which", "who", "whom", "this", "that", 
    "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", 
    "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", 
    "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", 
    "at", "by", "for", "with", "about", "against", "between", "into", "through", 
    "during", "before", "after", "above", "below", "to", "from", "up", "down", 
    "in", "out", "on", "off", "over", "under", "again", "further", "then", 
    "once", "here", "there", "when", "where", "why", "how", "all", "any", 
    "both", "each", "few", "more", "most", "other", "some", "such", "no", 
    "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", 
    "t", "can", "will", "just", "don", "should", "now", "d", "ll", "m", "o", 
    "re", "ve", "y", "ain", "aren", "couldn", "didn", "doesn", "hadn", "hasn", 
    "haven", "isn", "ma", "mightn", "mustn", "needn", "shan", "shouldn", 
    "wasn", "weren", "won", "wouldn"
]
##spaCy 
spacyStopwords = [
    "'d", "'ll", "'m", "'re", "'s", "'ve", "a", "about", "above", "across", "after", 
    "afterwards", "again", "against", "all", "almost", "alone", "along", "already", 
    "also", "although", "always", "am", "among", "amongst", "amount", "an", "and", 
    "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", 
    "around", "as", "at", "back", "be", "became", "because", "become", "becomes", 
    "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", 
    "besides", "between", "beyond", "both", "bottom", "but", "by", "ca", "call", 
    "can", "cannot", "could", "did", "do", "does", "doing", "done", "down", "due", 
    "during", "each", "eight", "either", "eleven", "else", "elsewhere", "empty", 
    "enough", "even", "ever", "every", "everyone", "everything", "everywhere", 
    "except", "few", "fifteen", "fifty", "first", "five", "for", "former", "formerly", 
    "forty", "four", "from", "front", "full", "further", "get", "give", "go", "had", 
    "has", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", 
    "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", 
    "hundred", "i", "if", "in", "indeed", "into", "is", "it", "its", "itself", "just", 
    "keep", "last", "latter", "latterly", "least", "less", "made", "make", "many", 
    "may", "me", "meanwhile", "might", "mine", "more", "moreover", "most", "mostly", 
    "move", "much", "must", "my", "myself", "n't", "name", "namely", "neither", 
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", 
    "not", "nothing", "now", "nowhere", "n‘t", "n’t", "of", "off", "often", "on", 
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", 
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps", "please", 
    "put", "quite", "rather", "re", "really", "regarding", "same", "say", "see", 
    "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", 
    "show", "side", "since", "six", "sixty", "so", "some", "somehow", "someone", 
    "something", "sometime", "sometimes", "somewhere", "still", "such", "take", 
    "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", 
    "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", 
    "they", "third", "this", "those", "though", "three", "through", "throughout", 
    "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", 
    "twenty", "two", "under", "unless", "until", "up", "upon", "us", "used", "using", 
    "various", "very", "via", "was", "we", "well", "were", "what", "whatever", 
    "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", 
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", 
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", 
    "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", 
    "‘d", "‘ll", "‘m", "‘re", "‘s", "‘ve", "’d", "’ll", "’m", "’re", "’s", "’ve"
]

index = create_index(pt_dataset.get_corpus_iter(), chatGPTStopwords)

bm25_chatGPTStopwords = pt.BatchRetrieve(index, wmodel="BM25")

index2 = create_index(pt_dataset.get_corpus_iter(),ntlkStopwords)
bm25_ntlkStopwords = pt.BatchRetrieve(index2, wmodel="BM25")

index3 = create_index(pt_dataset.get_corpus_iter(),spacyStopwords)
bm25_spacyStopwords = pt.BatchRetrieve(index3, wmodel="BM25")

ir-benchmarks/ir-acl-anthology-20240504-training documents:  71%|███████   | 90005/126958 [00:21<00:05, 6223.03it/s]

15:02:27.562 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-benchmarks/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:27<00:00, 4650.86it/s] 


15:02:36.248 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


ir-benchmarks/ir-acl-anthology-20240504-training documents:  71%|███████   | 90188/126958 [00:17<00:05, 6932.30it/s]

...

15:03:03.710 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


ir-benchmarks/ir-acl-anthology-20240504-training documents:  65%|██████▌   | 82855/126958 [00:14<00:07, 5589.87it/s]

15:03:20.971 [ForkJoinPool-3-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (1980.sigirconf_conference-80.24) - further warnings are suppressed


ir-benchmarks/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:21<00:00, 5898.75it/s] 


15:03:29.760 [ForkJoinPool-3-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 4 empty documents


### Evaluation

In [4]:
pt.Experiment(
    retr_systems=[bm25, bm25_chatGPTStopwords,bm25_ntlkStopwords,bm25_spacyStopwords],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['BM25', 'BM25_chatGPTStopwords','BM25_ntlkStopwords','BM25_spacyStopwords'],
    eval_metrics=['recall_1000', 'ndcg_cut_5', 'ndcg_cut.10', 'recip_rank']
)

,name,recall_1000,ndcg_cut_5,ndcg_cut.10,recip_rank
0,BM25,0.825376,0.393650,0.374041,0.579877
1,BM25_chatGPTStopwords,0.834410,0.382805,0.367656,0.581239
2,BM25_ntlkStopwords,0.834410,0.380311,0.366199,0.580013
3,BM25_spacyStopwords,0.834644,0.372564,0.359846,0.578987
